In [1]:
import pandas as pd
import os

from ast import literal_eval

In [2]:
DATA_PATH = os.path.join(
   '..', '..', '..', '..', "data", "frameworks_data", 'per_proj_data'
)

In [3]:
classification_df = pd.read_csv(os.path.join(DATA_PATH, '..', 'data_v0.7.1', 'full_dataset_with_translations.csv'))
classification_afs = classification_df.analysis_framework_id.unique()
classification_projects = classification_df.project_id.unique()

len(classification_afs), len(classification_projects)

/home/selim/anaconda3/envs/deepl/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(13, 63)

In [18]:
afs_df = pd.read_csv(os.path.join(DATA_PATH, 'full_AF.csv')).drop_duplicates()

turkey_df = pd.read_csv(os.path.join(DATA_PATH, '2021_ifrc_turkey_v0.1.csv')).drop_duplicates()
ukraine_df = pd.read_csv(os.path.join(DATA_PATH, 'ukraine_v0.1.csv')).drop_duplicates()


In [19]:
afs_df = afs_df[['id', 'title']].rename(columns={'id': 'analysis_framework_id', 'title': 'Framework Title'})
afs_df

,analysis_framework_id,Framework Title
0,28,DRC Test Event
1,13,Sample Framework
2,166,Sample Framework (cloned)
3,29,Somalia Test Event
4,473,_aNRC Afghanistan test framework (cloned) (clo...
...,...,...
1741,2078,JIAF Test July 17 Interns (cloned)
1742,2079,AI & ML Framework
1743,2080,My Research Framework
1744,2081,Qualitative Data Analysis Saudi Youth NEET


In [22]:
all_projects = pd.read_csv(os.path.join(DATA_PATH, 'projects_to_be_chosen_for_NLP_models.csv'))
all_projects.head(2)

,project_id,AF_id,first_name,last_name,email,title,description,# of entries,# of leads,AF already mapped,project in current training data,Hum-sector-models,development-sector-models,programme-evaluation-models
0,2587,1623,Patrice,Chataigner,pc@okularanalytics.com,2021 IFRC ESSN Turkey,NaN,35943,2825,False,False,True,False,False
1,3019,1918,Patrice,Chataigner,pc@okularanalytics.com,2022 IMAC Ukraine,Ukraine situation analysis IMAC,18705,4919,False,False,True,False,False


In [23]:
list_hum_afs = all_projects[all_projects['Hum-sector-models']].AF_id.unique()
len(list_hum_afs)

80

In [134]:
list_dev_afs = all_projects[all_projects['development-sector-models']].AF_id.unique().tolist()
list_dev_afs.remove(699)
len(list_dev_afs)

1

In [135]:
from ast import literal_eval

In [136]:
all_projs = pd.read_csv(os.path.join(DATA_PATH, 'all_projects_summary.csv')).drop(
    columns=[
        "is_private",
        "start_date",
        "end_date",
        "has_publicly_viewable_unprotected_leads",
        "has_publicly_viewable_confidential_leads",
        "has_publicly_viewable_restricted_leads",
        'created_at',
        'created_by_id',
        'is_visualization_enabled',
        'status'
    ]
)
hum_afs_df = all_projs[all_projs.analysis_framework_id.isin(list_hum_afs)][['analysis_framework_id', 'primary_tags_structure']].drop_duplicates()
hum_afs_df = pd.merge(left=afs_df, right=hum_afs_df, how='right', on='analysis_framework_id')
hum_afs_df

,analysis_framework_id,Framework Title,primary_tags_structure
0,1623,IFRC ESSN Turkey 2021,{'1d': {'Operational Environment': {'1dpillars...
1,1918,IMAC Ukraine Analysis Framework,{'1d': {'Operational Environment': {'1dpillars...
2,1306,IMMAP/DFS Situation Analysis Framework,{'1d': {'Operational Environment': {'1dpillars...
3,699,IFRC Master Framework 2019,"{'1d': {'Flag': {'1dpillars': ['Flag'], '1dsub..."
4,829,Colombia-AF,{'1d': {'Operational Environment': {'1dpillars...
...,...,...,...
75,1359,GIMAC,{'1d': {'HIGH LEVEL TAGS': {'1dpillars': ['Gen...
76,706,IFRC Argentina,"{'1d': {'Flag': {'1dpillars': ['Flag'], '1dsub..."
77,710,IFRC Belarus,{'1d': {'Operational Environment': {'1dpillars...
78,701,IFRC Afghanistan,{'1d': {'Operational Environment': {'1dpillars...


In [138]:
def return_final_empty_mapping_df(df):

    final_mapping_df = pd.DataFrame()

    for index, row in df.iterrows():
        prim_tags = literal_eval(row.primary_tags_structure)

        #1D
        tags_1d = prim_tags["1d"]
        if len(tags_1d) > 0:
            for original_type, pillar_subpillar in tags_1d.items():
                
                if "operational environment" == original_type.lower():
                    type_name = "1D"

                else:
                    type_name = original_type

                for subpillars in pillar_subpillar['1dsubpillars']:
                    split_subpilllars = subpillars.split('->')

                    row = pd.DataFrame(
                        {
                            "analysis_framework_id": row.analysis_framework_id,
                            "Framework Title": row['Framework Title'],
                            "Original Type": type_name.lower(),
                            "Original first level": split_subpilllars[0].lower(),
                            "Original second level": split_subpilllars[1].lower(),
                        }, index=[0]
                    )

                    final_mapping_df = final_mapping_df.append(row)

        # 2D
        tags_2d = prim_tags["2d"]
        if len(tags_2d) > 0:
            for k, v in tags_2d.items():
                for original_type, pillar_subpillar in v.items():
                    if original_type.lower() == '2dsubpillars':
                        for subpillars in pillar_subpillar:
                            type_name = '2D'
                            split_subpilllars = subpillars.split('->')

                            row = pd.DataFrame(
                                {
                                    "analysis_framework_id": row.analysis_framework_id,
                                    "Framework Title": row['Framework Title'],
                                    "Original Type": type_name,
                                    "Original first level": split_subpilllars[0].lower(),
                                    "Original second level": split_subpilllars[1].lower(),
                                }, index=[0]
                            )

                            final_mapping_df = final_mapping_df.append(row)

                    if original_type.lower() == 'subsectors':
                        for subpillars in pillar_subpillar:
                            type_name = 'subsectors'
                            split_subpilllars = subpillars.split('->')

                            row = pd.DataFrame(
                                {
                                    "analysis_framework_id": row.analysis_framework_id,
                                    "Framework Title": row['Framework Title'],
                                    "Original Type": type_name,
                                    "Original first level": split_subpilllars[0].lower(),
                                    "Original second level": split_subpilllars[1].lower(),
                                }, index=[0]
                            )

                            final_mapping_df = final_mapping_df.append(row)

                    
    final_mapping_df = final_mapping_df.drop_duplicates().sort_values(by=['Original first level', 'Original second level']).drop_duplicates()

    for col in ['NLP Type',	'NLP first level', 'NLP second level', "First-pass", "Reversible"]:
        final_mapping_df[col] = ''

    return final_mapping_df

final_mapping_df_hum = return_final_empty_mapping_df(hum_afs_df)

In [ ]:
final_mapping_df_hum.head(2)

,analysis_framework_id,Framework Title,Original Type,Original first level,Original second level,NLP Type,NLP first level,NLP second level,First-pass,Reversible
0,1883,Situational Awareness Overview Eastern Caribbean,shocks and their impacts,2017 hurricane season,access & logistics challenges,,,,,
0,1883,Situational Awareness Overview Eastern Caribbean,shocks and their impacts,2017 hurricane season,displacement,,,,,


In [102]:
first_release_mapping = pd.read_csv(os.path.join(DATA_PATH,"mapping_first_release.csv" )).drop(columns=['Cover'])
first_release_mapping['Original Pillar'] = first_release_mapping['Original Pillar'].str.lower()
first_release_mapping['Original Sub-pillar'] = first_release_mapping['Original Sub-pillar'].str.lower()
first_release_mapping.loc[first_release_mapping['Framework Title']== 'JIAF Multi Sectoral', 'Framework Title'] = 'JIAF multi sectoral'
first_release_mapping = pd.merge(left=afs_df, right=first_release_mapping, how='right', on='Framework Title').rename(
    columns={
        "NLP Pillar": "NLP first level",
        "NLP Sub-pillar": "NLP second level",
        "Original Pillar": "Original first level",
        "Original Sub-pillar": "Original second level",
    }
)

first_release_mapping.head(3)

,analysis_framework_id,Framework Title,Original Type,Original first level,Original second level,NLP Type,NLP first level,NLP second level,First-pass,Verified,Reversible
0,1623,IFRC ESSN Turkey 2021,2D,at risk,people at risk / vulnerable,2D,At Risk,Risk and vulnerabilities,YES,NO,YES
1,1306,IMMAP/DFS Situation Analysis Framework,2D,at risk,people at risk / vulnerable,2D,At Risk,Risk and vulnerabilities,YES,NO,YES
2,1700,JIAF multi sectoral,2D,at risk,people at risk / vulnerable,2D,At Risk,Risk and vulnerabilities,YES,NO,YES


In [115]:
first_release_mapping.shape

(477, 11)

In [116]:
first_release_mapping[first_release_mapping['Original first level']=='at risk']

,analysis_framework_id,Framework Title,Original Type,Original first level,Original second level,NLP Type,NLP first level,NLP second level,First-pass,Verified,Reversible
0,1623,IFRC ESSN Turkey 2021,2D,at risk,people at risk / vulnerable,2D,At Risk,Risk and vulnerabilities,YES,NO,YES
1,1306,IMMAP/DFS Situation Analysis Framework,2D,at risk,people at risk / vulnerable,2D,At Risk,Risk and vulnerabilities,YES,NO,YES
2,1700,JIAF multi sectoral,2D,at risk,people at risk / vulnerable,2D,At Risk,Risk and vulnerabilities,YES,NO,YES


In [122]:
mapping_subsectors = pd.read_csv(os.path.join(DATA_PATH, '..', 'subsectors', 'mapping_all_afs_to_nlp - sectors_subsectors.csv')).drop(columns=['Unnamed: 5', '# occurences'])
for col in mapping_subsectors.columns.tolist():
    mapping_subsectors[col] = mapping_subsectors[col].str.lower()
mapping_subsectors = mapping_subsectors.dropna()
mapping_subsectors.head(2)

,Original sector,Original subsector,NLP sector,NLP subsector
0,education,facilities and amenities,education,learning environment
1,education,learning environment,education,learning environment


In [124]:
def clean_str(entry):
    return entry.replace(" ", "")

In [126]:
mapping_sheet_filled = pd.DataFrame()

for index, row in final_mapping_df_hum.iterrows():

    row_first_lvl = clean_str(row["Original first level"])
    row_second_lvl = clean_str(row["Original second level"])

    subsectors_mapped_tmp = mapping_subsectors[
        (mapping_subsectors["Original sector"].apply(clean_str) == row_first_lvl)
        & (mapping_subsectors["Original subsector"].apply(clean_str) == row_second_lvl)
    ]

    first_release_tmp = first_release_mapping[
        (first_release_mapping["Original first level"].apply(clean_str) == row_first_lvl)
        & (
            first_release_mapping["Original second level"].apply(clean_str)
            == row_second_lvl
        )
    ]

    if len(subsectors_mapped_tmp) == 1:
        row["NLP Type"] = "subsectors"
        row["First-pass"] = True
        row["NLP first level"] = subsectors_mapped_tmp["NLP sector"].values[0]
        row["NLP second level"] = subsectors_mapped_tmp["NLP subsector"].values[0]

    if len(first_release_tmp) >= 1:
        row["NLP Type"] = first_release_tmp["NLP Type"].values[0]
        row["First-pass"] = True
        row["NLP first level"] = first_release_tmp["NLP first level"].values[0]
        row["NLP second level"] = first_release_tmp["NLP second level"].values[0]

    mapping_sheet_filled = mapping_sheet_filled.append(row)

mapping_sheet_filled = mapping_sheet_filled[
    [
        "analysis_framework_id",
        "Framework Title",
        "Original Type",
        "Original first level",
        "Original second level",
        "NLP Type",	
        "NLP first level", 
        "NLP second level",
        "First-pass",
        "Reversible"
    ]
]


In [127]:
mapping_sheet_filled['First-pass'].apply(lambda x: type(x) is bool).sum()

1814

In [128]:
mapping_sheet_filled.to_csv('mapping_sheet_all_to_be_filled.csv', index=None)
mapping_sheet_filled

,analysis_framework_id,Framework Title,Original Type,Original first level,Original second level,NLP Type,NLP first level,NLP second level,First-pass,Reversible
0,1883.0,Situational Awareness Overview Eastern Caribbean,shocks and their impacts,2017 hurricane season,access & logistics challenges,,,,,
0,1883.0,Situational Awareness Overview Eastern Caribbean,shocks and their impacts,2017 hurricane season,displacement,,,,,
0,1883.0,Situational Awareness Overview Eastern Caribbean,shocks and their impacts,2017 hurricane season,impacts and needs,,,,,
0,1883.0,Situational Awareness Overview Eastern Caribbean,shocks and their impacts,2017 hurricane season,lessons learned,,,,,
0,1883.0,Situational Awareness Overview Eastern Caribbean,shocks and their impacts,2017 hurricane season,most affected geographic areas,,,,,
...,...,...,...,...,...,...,...,...,...,...
0,237.0,SoHSS,meta-data,year,2015,,,,,
0,237.0,SoHSS,meta-data,year,2016,,,,,
0,237.0,SoHSS,meta-data,year,2017,,,,,
0,237.0,SoHSS,meta-data,year,2018,,,,,


## DEV SECTOR

In [139]:
dev_afs_df = all_projs[all_projs.analysis_framework_id.isin(list_dev_afs)][['analysis_framework_id', 'primary_tags_structure']].drop_duplicates()
dev_afs_df = pd.merge(left=afs_df, right=dev_afs_df, how='right', on='analysis_framework_id')
dev_afs_df

,analysis_framework_id,Framework Title,primary_tags_structure
0,1706,CCA Framework - Option 2,{'1d': {'High Level Tags': {'1dpillars': ['Hig...


In [140]:
final_mapping_df_dev = return_final_empty_mapping_df(dev_afs_df)

In [141]:
final_mapping_df_dev.head(30)

,analysis_framework_id,Framework Title,Original Type,Original first level,Original second level,NLP Type,NLP first level,NLP second level,First-pass,Reversible
0,1706,CCA Framework - Option 2,2D,covid-19 impact,negative coping strategies,,,,,
0,1706,CCA Framework - Option 2,2D,covid-19 impact,socio-economic impact,,,,,
0,1706,CCA Framework - Option 2,2D,covid-19 impact,status of recovery,,,,,
0,1706,CCA Framework - Option 2,high level tags,high level tags,capacities,,,,,
0,1706,CCA Framework - Option 2,high level tags,high level tags,challenges / opportunities towards 2030 agenda,,,,,
0,1706,CCA Framework - Option 2,high level tags,high level tags,information gaps,,,,,
0,1706,CCA Framework - Option 2,high level tags,high level tags,progress towards 2030 agenda,,,,,
0,1706,CCA Framework - Option 2,high level tags,high level tags,recommendations from stakeholders,,,,,
0,1706,CCA Framework - Option 2,2D,progress towards 2030 agenda,achievements / situational snapshot,,,,,
0,1706,CCA Framework - Option 2,2D,progress towards 2030 agenda,challenges & barriers,,,,,
